# ZenML: Open-source MLOps Framework for reproducible ML pipelines

![Test](_assets/Logo/zenml.svg)

<div class="alert alert-block alert-danger">
    <b>Note:</b> This lesson is still in progress and some commands may not work as described. Please expect an update until 20th April 2022. 
</div>

In [ ]:
from absl import logging as absl_logging
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
absl_logging.set_verbosity(-10000)

Let's begin by initializing ZenML in our directory. We are going to use a local stack to begin with, for simplicity and then transition to other stacks. This can be achieved in code by executing the following block.

# Initialize ZenML

In [ ]:
!rm -rf .zen
!zenml init

# Install integrations

ZenML handles integrations natively, to avoid dependency conflicts, so make sure to use the following command to install the integrations required for this lesson.

![All](_assets/integrations_all.png "All")

In [ ]:
!zenml integration install kubeflow seldon aws -f

# The Concept of MLOps Stacks

The ZenML stack is a concept that describes the union of Metadata Store, Artifact Store and Orchestrator that will be used for all pipeline runs. When you get started with zenml you start off with a default local stack.

In [ ]:
!zenml stack list

## The Local Stack

You can imagine the local stack to look like this. Within the diagram we show how a generic pipeline interacts with the local stack.

![LocalStack](_assets/localstack.png "LocalStack")

## The Kubeflow Pipelines stack

We will now use the Kubeflow integration to extend the concept of stacks

Now we want to transition to a kubeflow stack that will look a little bit like this. Note that for kubeflow pipelines we also need a registry where the docker images for each step are registered. 

![KubeflowStack](_assets/aws_stack_redesigned.png "KubeflowStack")

But we have good news! You barely have to do anything to transition.

# Transitioning to Production with Kubeflow on AWS

There are two steps to follow in order to continue.

- Set up the neccessary cloud resources on the provider of your choice
- Configure ZenML with a new stack to be able to communicate with these resources

## Set up using the cloud guide

In order to continue, it is best to follow the updated cloud guide for ZenML found [here](https://docs.zenml.io/features/guide-aws-gcp-azure). Please return after finishing the `pre-requisites` section.

It is recommended you use AWS as your cloud provider to follow along the lesson. However, if you were to select GCP or Azure, it should not so hard to actually modify the below commands to work accordingly.

## Create your AWS Kubeflow Stack

Now we can configure a new stack that points to your newly created resources on the cloud

If you remember from the main README, Kubernetes and Docker are a pre-requisite to this part of the guide. Please make sure you have them installed. You also need to install the [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) to move forward.

In [ ]:
# Replace the following with your own configuration. Use the below as exemplary.

AWS_EKS_CLUSTER="zenhacks-cluster"
AWS_REGION="us-east-1"
ECR_REGISTRY_NAME="715803424590.dkr.ecr.us-east-1.amazonaws.com"
S3_BUCKET_NAME="s3://zenbytes-bucket"

In [ ]:
# Register container registry
# Point Docker to the registry
!aws ecr get-login-password --region {AWS_REGION} | docker login --username AWS --password-stdin {ECR_REGISTRY_NAME}
!zenml container-registry register ecr_registry --type=default --uri={ECR_REGISTRY_NAME}

# Register orchestrator (Kubeflow on AWS)
# if you don't have your kubectl pointing to the eks cluster, run this
!aws eks --region {AWS_REGION} update-kubeconfig --name {AWS_EKS_CLUSTER}
!zenml orchestrator register eks_orchestrator --type=kubeflow 

# Register metadata store and artifact store
!zenml metadata-store register kubeflow_metadata_store --type=kubeflow
!zenml artifact-store register s3_store --type=s3 --path={S3_BUCKET_NAME}


# Register a secret manager
!zenml secrets-manager register local_secret_manager --type=local

# Register the aws_kubeflow_stack
!zenml stack register aws_kubeflow_stack -m kubeflow_metadata_store -a s3_store -o eks_orchestrator -c ecr_registry -x local_secret_manager

## Transition to Production (Run on the Cloud)

Once the stack is configured, all that is left to do is to set it active and to run a pipeline. Note that the code itself DOES NOT need to change, only the active stack.

ZenML will detect that the stack has changed, and instead of running your pipeline locally, will build a Docker Image, push it to the container registry with your requirements, and deploy the pipeline with that image on Kubeflow Pipelines. This whole process is usually very painful but simplified with ZenML, and is completely customizable.

For now, try it out! It might take a few minutes to build and push the image, but after that you'd see your pipeline in the cloud!

<div class="alert alert-block alert-info">
    <b>Note:</b> Currently running pipelines defined within a jupyter notebook cell is
    not supported. To get around this you can run the train pipeline within this repo. 
</div>

In [ ]:
!zenml stack set aws_kubeflow_stack

# Let's train within kubeflow pipelines - this will deploy the pipeline in a one of manner
!python run.py --deploy --kubernetes-context=arn:aws:eks:us-east-1:715803424590:cluster/zenhacks-cluster --namespace=kubeflow --base-url=http://abb84c444c7804aa98fc8c097896479d-377673393.us-east-1.elb.amazonaws.com --interval-second=300 

In order to see the pipeline run, you should port-forward Kubeflow Pipelines to: [http://localhost:8080/](http://localhost:8080/). You might want to try this is a seperate shell:

```
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80
```

In [ ]:
# Do this only if the port forward from `zenml stack up` did not work. 
!kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80

And thats it you have successfully transitioned from local to production by simply switching you ZenML stack. This is just scratching the surface!

Next up, more about stacks, running pipelines on a schedule, and much more coming soon!